# GameZone

### Console Wars

### Author: Wenyi Xu

Discuss the epic console war amongst **Sony**, **Microsoft** and **Nintendo**.

Scape to get some datasets for analysis and visualization to show the sales and other statistics for different platforms.

## Backgroud Information

### Playstation Release timeline

* PS1 Release Date	Saturday - December 3, 1994

* PS2 Release Date	Saturday - March 4, 2000

* PS3 Release Date	Saturday - November 11, 2006

* PS4 Release Date	Friday - Nov 15, 2013 

* PS4 Pro Release	Thursday - Nov 10, 2016 

### Playstation Release timeline

* Xbox Release Date	- November 15, 2001

* Xbox 360 Release Date	- November 22, 2005

* Xbox One Release Date	- November 22, 2013

* Xbox One X Release Date - June 13, 2016

### Nintendo Platforms release timeline

* NES Release Date - July 15, 1983

* Nintendo 64 Release Date - June 23, 1996

* GameCube Release Date - September 14, 2001

* Wii Release Date - November 19, 2006

* Wii u Release Date - November 18, 2012

* Switch Release Date - March 3, 2017


## Part 0: Scrape VGChartz

Scrape VGChartz to get useful datasets about game and hardware sales for different platforms.

I can only find sales data of home video games consoles on VGChartz starting from 25th Nov 2006 (PS3 & XBOX360 & Wii), so the data of PS2 and other previous consoles need to be extracted from other sources.

In [31]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

In [178]:
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
from datetime import datetime
from datetime import timedelta
import math
import time

The VGChartz important pages that contain data we need

In [32]:
# Main page
vgchartz_main_url = 'http://www.vgchartz.com/'
# Gloabl weekly sales
# http://www.vgchartz.com/weekly/[date_id]/Global/
# date_id: start from 39047 (25th Nov 2006) to 43058 (18th Nov 2017)
date_id_start = 39047
date_start = datetime(2006, 11, 25)
date_id_end = 43058
date_end = datetime(2017, 11, 18)

date_id = 39047

vgchartz_weekly_url_head = 'http://www.vgchartz.com/weekly/'
vgchartz_weekly_url_tail = '/Global/'
vgchartz_weekly_url = vgchartz_weekly_url_head + str(date_id) + vgchartz_weekly_url_tail

Check if the date diff correspond to the date_id diff

In [27]:
print(datetime(2017, 11, 18) - date_start)
print(43058 - 39047)

4011 days, 0:00:00
4011


### Function get_weekly_url

**Date --> weekly_url**

We have

* Start date: date_start (25th Nov 2006)

* Start date id: date_id_start (39047)

* End date: date_end (18th Nov 2017)

* End date id: date_id_end (43058)

When given a random date between 25th Nov 2006 & 18th Nov 2017, we want to first get its corresponding week_id, and then generate its vgchartz_weekly_url


In [47]:
# date: in datetime format
def get_weekly_url(date):
    
    if type(date) != type(date_end):
        return 'Date must be type datetime'
    if date < date_start or date > date_end:
        return 'Date must be from 25th Nov 2006 to 18th Nov 2017'
    
    date_diff = date - date_start
    day_diff = date_diff.days
    # Round up
    week_diff = math.ceil(day_diff/7.0)
    date_id = date_id_start + week_diff*7
    
    vgchartz_weekly_url = vgchartz_weekly_url_head + str(date_id) + vgchartz_weekly_url_tail
    return vgchartz_weekly_url

In [52]:
# Test the function get_weekly_url
print(get_weekly_url('25th Nov 2006') == 'Date must be type datetime')
print(get_weekly_url(datetime(2000, 11, 20)) == 'Date must be from 25th Nov 2006 to 18th Nov 2017')
print(get_weekly_url(datetime(2017, 11, 20)) == 'Date must be from 25th Nov 2006 to 18th Nov 2017')
print(get_weekly_url(datetime(2007, 2, 17)) == 'http://www.vgchartz.com/weekly/39131/Global/')

True
True
True
True


Create the PoolManager

In [12]:
http = urllib3.PoolManager()

Make the request 

and query the pages

In [65]:
vgchartz_main_response = http.request('GET', vgchartz_main_url)
vgchartz_main_soup = BeautifulSoup(response.data, "lxml")

vgchartz_week_response = http.request('GET', vgchartz_weekly_url)
vgchartz_week_soup = BeautifulSoup(vgchartz_week_response.data, "lxml")

Now we have **soup**: the **HTML** of the VGCHartz pages.

Now try to extract some useful data.

There are 2 useful tables in weekly sales pages: **Global Hardware by Platform** & **Global Software by Platform**

Example of the **Global Hardware by Platform** table:

|Platform |	Weekly (change) |	Total |
|---------|-----------------|--------|
|DS	| 905,597	(+88%) |	29,319,098 |
|Wii |	529,658	(N/A) |	529,658 |
|X360 |	361,561	(+87%) |	5,943,800 |
|PSP |	352,884	(+87%) |	17,124,390 |
|PS3 |	103,130	(-50%) |	394,937 |


In [113]:
tags = vgchartz_week_soup.find_all('h2', attrs={'class': 'heading'})
tags

[<h2 class="heading">Global Hardware by Platform</h2>,
 <h2 class="heading">Global Software by Platform</h2>]

Extract the table names from the corresponding tags

In [112]:
table_names = [tag.text for tag in tags]   
table_names

['Global Hardware by Platform', 'Global Software by Platform']

### Extract the Global Hardware by Platform table

In [123]:
charts = vgchartz_week_soup.find_all('table', class_="chart")
len(charts)

4

After manually check the HTML, I found there are totally 4 charts per page.

And the 'Global Hardware by Platform' is the 3rd one -_-

In [126]:
hardware_sale_table = charts[2]
hardware_sale_table.find_all('th')

[<th align="left">Platform</th>,
 <th align="right" colspan="2">Weekly <span style="font-size:70%;">(change)</span></th>,
 <th align="right">Total</th>,
 <th align="left">Total</th>,
 <th align="right" style="padding-right:1px;">2,252,830</th>,
 <th align="left" style="padding-left:2px;"><span style="font-size:70%;">(+111%)</span>
 </th>,
 <th align="left"> </th>]

Get all the **colume** names

In [128]:
hardware_sale_cols = [x.text for x in hardware_sale_table.find_all('th')][:3]
hardware_sale_cols

['Platform', 'Weekly (change)', 'Total']

In [137]:
hardware_sale_table.find_all('td', attrs={})

[<td align="left">DS</td>,
 <td align="right" style="padding-right:1px;">905,597</td>,
 <td align="left" style="padding-left:2px;"><span style="font-size:70%;">(+88%)</span></td>,
 <td align="right">29,319,098</td>,
 <td align="left">Wii</td>,
 <td align="right" style="padding-right:1px;">529,658</td>,
 <td align="left" style="padding-left:2px;"><span style="font-size:70%;">(N/A)</span></td>,
 <td align="right">529,658</td>,
 <td align="left">X360</td>,
 <td align="right" style="padding-right:1px;">361,561</td>,
 <td align="left" style="padding-left:2px;"><span style="font-size:70%;">(+87%)</span></td>,
 <td align="right">5,943,800</td>,
 <td align="left">PSP</td>,
 <td align="right" style="padding-right:1px;">352,884</td>,
 <td align="left" style="padding-left:2px;"><span style="font-size:70%;">(+87%)</span></td>,
 <td align="right">17,124,390</td>,
 <td align="left">PS3</td>,
 <td align="right" style="padding-right:1px;">103,130</td>,
 <td align="left" style="padding-left:2px;"><span

In [140]:
hardware_sale_rows = hardware_sale_table.find_all('td', attrs={})
hardware_sale_rows = [row.text.strip() for row in hardware_sale_rows]
hardware_sale_rows

['DS',
 '905,597',
 '(+88%)',
 '29,319,098',
 'Wii',
 '529,658',
 '(N/A)',
 '529,658',
 'X360',
 '361,561',
 '(+87%)',
 '5,943,800',
 'PSP',
 '352,884',
 '(+87%)',
 '17,124,390',
 'PS3',
 '103,130',
 '(-50%)',
 '394,937']

### Function get_weekly_hardware_sales

**Date --> weekly hardware sales data**

Given a date from 25th Nov 2006 to 18th Nov 2017, we want to get which week this date is in, and get its corresponding weekly hardware sales data

In [149]:
# date: in datetime format
def get_weekly_hardware_sales(date):
    
    # Get the corresponding weekly_url from input date
    weekly_url = get_weekly_url(date)
    # Create the pool manager
    pool_manager = urllib3.PoolManager()
    
    weekly_response = pool_manager.request('GET', weekly_url)
    weekly_soup = BeautifulSoup(weekly_response.data, "lxml")
    
    # Get the weekly hardware sales table
    weekly_charts = weekly_soup.find_all('table', class_="chart")
    weekly_hardware_sale_table = weekly_charts[2]
    # Get the colume names
    # Columes should be 'Platforms', 'Weekly Sales', 'Weekly Sales Changes', 'Total Sales'
    platforms = []
    weekly_sales = []
    weekly_sales_changes = []
    total_sales = []
    weekly_hardware_sale_cols = [x.text for x in weekly_hardware_sale_table.find_all('th')][:3]
    # Get the rows
    weekly_hardware_sale_rows = weekly_hardware_sale_table.find_all('td', attrs={})
    weekly_hardware_sale_rows = [row.text.strip() for row in weekly_hardware_sale_rows]   
    # Distribute data of diff fields into diff lists
    for i in range(len(weekly_hardware_sale_rows)):
        if i%4 == 0:
            platforms.append(weekly_hardware_sale_rows[i])
        elif i%4 == 1:
            weekly_sales.append(weekly_hardware_sale_rows[i])
        elif i%4 == 2:
            weekly_sales_changes.append(weekly_hardware_sale_rows[i])
        else:
            total_sales.append(weekly_hardware_sale_rows[i])
            
    weekly_hardware_sales_df = pd.DataFrame({'Platforms': platforms, \
                                      'Weekly Sales': weekly_sales, \
                                      'Weekly Sales Changes': weekly_sales_changes, \
                                      'Total Sales': total_sales
                                     })
    return weekly_hardware_sales_df

In [150]:
date = datetime(2007, 11, 20)
get_weekly_hardware_sales(date)

,Platforms,Total Sales,Weekly Sales,Weekly Sales Changes
0,DS,"56,338,561","1,353,850",(+105%)
1,Wii,"15,192,774","673,069",(+46%)
2,PSP,"29,233,466","446,202",(+60%)
3,X360,"13,704,598","414,403",(+82%)
4,PS3,"6,638,792","376,441",(+46%)


## Part 1: Analyze the Weekly Sales

#### Wii Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [184]:
start_date = datetime(2006, 11, 25)
end_date = datetime(2017, 11, 18)

date_p = start_date
wii_sales_date = []
wii_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if Wii is in sale
    if 'Wii' in weekly_hardware_sales['Platforms'].values:
        wii_sales_date.append(date_p)
        wii_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='Wii']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.5)

In [186]:
wii_sales_df = pd.DataFrame({'Date': wii_sales_date, \
                             'Total Sales': wii_sales \
                                     })
wii_sales_df[-10:]

,Date,Total Sales
466,2015-10-31,"100,998,916"
467,2015-11-07,"100,999,541"
468,2015-11-14,"101,000,343"
469,2015-11-21,"101,001,662"
470,2015-11-28,"101,003,840"
471,2015-12-05,"101,005,477"
472,2015-12-12,"101,007,242"
473,2015-12-19,"101,009,240"
474,2015-12-26,"101,011,107"
475,2016-01-02,"101,011,951"


#### PS3 Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [187]:
start_date = datetime(2006, 11, 25)
end_date = datetime(2017, 11, 18)

date_p = start_date
ps3_sales_date = []
ps3_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if Wii is in sale
    if 'PS3' in weekly_hardware_sales['Platforms'].values:
        ps3_sales_date.append(date_p)
        ps3_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='PS3']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [188]:
PS3_sales_df = pd.DataFrame({'Date': ps3_sales_date, \
                             'Total Sales': ps3_sales \
                                     })
PS3_sales_df[-10:]

,Date,Total Sales
564,2017-09-16,"86,892,414"
565,2017-09-23,"86,893,083"
566,2017-09-30,"86,893,656"
567,2017-10-07,"86,894,195"
568,2017-10-14,"86,894,717"
569,2017-10-21,"86,895,166"
570,2017-10-28,"86,895,649"
571,2017-11-04,"86,896,170"
572,2017-11-11,"86,896,630"
573,2017-11-18,"86,897,152"


#### XBOX 360 Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [189]:
start_date = datetime(2006, 11, 25)
end_date = datetime(2017, 11, 18)

date_p = start_date
x360_sales_date = []
x360_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if Wii is in sale
    if 'X360' in weekly_hardware_sales['Platforms'].values:
        x360_sales_date.append(date_p)
        x360_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='X360']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [190]:
x360_sales_df = pd.DataFrame({'Date': x360_sales_date, \
                             'Total Sales': x360_sales \
                                     })
x360_sales_df[-10:]

,Date,Total Sales
564,2017-09-16,"85,807,263"
565,2017-09-23,"85,807,742"
566,2017-09-30,"85,808,186"
567,2017-10-07,"85,808,639"
568,2017-10-14,"85,809,123"
569,2017-10-21,"85,809,603"
570,2017-10-28,"85,810,055"
571,2017-11-04,"85,810,486"
572,2017-11-11,"85,810,862"
573,2017-11-18,"85,811,280"


#### Nintendo Switch Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [191]:
start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 11, 18)

date_p = start_date
ns_sales_date = []
ns_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if Wii is in sale
    if 'NS' in weekly_hardware_sales['Platforms'].values:
        ns_sales_date.append(date_p)
        ns_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='NS']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [192]:
ns_sales_df = pd.DataFrame({'Date': ns_sales_date, \
                             'Total Sales': ns_sales \
                                     })
ns_sales_df[-10:]

,Date,Total Sales
28,2017-09-10,"5,925,303"
29,2017-09-17,"6,081,881"
30,2017-09-24,"6,282,240"
31,2017-10-01,"6,437,927"
32,2017-10-08,"6,605,881"
33,2017-10-15,"6,770,056"
34,2017-10-22,"7,094,564"
35,2017-10-29,"7,329,519"
36,2017-11-05,"7,606,357"
37,2017-11-12,"7,994,004"


#### PS4 Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [193]:
start_date = datetime(2013, 6, 1)
end_date = datetime(2017, 11, 18)

date_p = start_date
ps4_sales_date = []
ps4_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if PS4 is in sale
    if 'PS4' in weekly_hardware_sales['Platforms'].values:
        ps4_sales_date.append(date_p)
        ps4_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='PS4']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [195]:
ps4_sales_df = pd.DataFrame({'Date': ps4_sales_date, \
                             'Total Sales': ps4_sales \
                                     })
ps4_sales_df[-10:]

,Date,Total Sales
200,2017-09-16,"64,352,281"
201,2017-09-23,"64,609,497"
202,2017-09-30,"64,909,187"
203,2017-10-07,"65,177,958"
204,2017-10-14,"65,442,943"
205,2017-10-21,"65,750,221"
206,2017-10-28,"66,031,358"
207,2017-11-04,"66,490,554"
208,2017-11-11,"67,170,279"
209,2017-11-18,"67,977,292"


#### XBOX ONE Sales Over Time

From 25th Nov 2006 to 18th Nov 2017

In [196]:
start_date = datetime(2013, 8, 17)
end_date = datetime(2017, 11, 18)

date_p = start_date
xone_sales_date = []
xone_sales = []

while date_p <= end_date:
    
    weekly_hardware_sales = get_weekly_hardware_sales(date_p)
    # Check if PS4 is in sale
    if 'XOne' in weekly_hardware_sales['Platforms'].values:
        xone_sales_date.append(date_p)
        xone_sales.append(weekly_hardware_sales[weekly_hardware_sales['Platforms']=='XOne']['Total Sales'].values[0])
    
    date_p += timedelta(7)
    time.sleep(0.1)

In [197]:
xone_sales_df = pd.DataFrame({'Date': xone_sales_date, \
                             'Total Sales': xone_sales \
                                     })
xone_sales_df[-10:]

,Date,Total Sales
199,2017-09-16,"30,989,732"
200,2017-09-23,"31,070,354"
201,2017-09-30,"31,176,640"
202,2017-10-07,"31,270,019"
203,2017-10-14,"31,364,608"
204,2017-10-21,"31,464,018"
205,2017-10-28,"31,573,460"
206,2017-11-04,"31,733,475"
207,2017-11-11,"32,562,278"
208,2017-11-18,"33,017,552"
